### Funciones adaptador de BD Censo a Modelos EPH

In [4]:
import pandas as pd

# Funcion para adaptar las respuestas del censo a las de la EPH
def adapta_rtas_a_EPH(table):
    """
    table: dataframe con las respuestas del censo
    """

    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    ## VIVIENDA
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    ## HOGAR
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
    # PERSONA
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

    return table

# Informacion accesoria 
# Departamento, Regiones.

dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')

# Ranking de Aglomerados y Regiones
AGLO_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/AGLO_rk')
rk_table = AGLO_rk.set_index(['ANO4', 'AGLOMERADO']).unstack()
AGLO_rk = rk_table.fillna(rk_table.mean()).stack().reset_index()

Reg_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/Reg_rk')

# Funcion para adaptar las muestras de censo a la EPH
def adapta_muestras_a_EPH(table):
    """
    table: dataframe con las respuestas del censo
    """
    # Adaptamos las categorias de respuestas para que sean iguales las de la EPH
    table = adapta_rtas_a_EPH(table)

    ## Agregar Region
    table = table.merge(dpto_region[['DPTO', 'Region']])

    ## Agregar ranking de Region y Aglo
    table = table.merge(AGLO_rk[['AGLOMERADO', 'ANO4', 'AGLO_rk']]).merge(Reg_rk[['Region', 'ANO4', 'Reg_rk']])

    return table

### Lista de archivos extraidos de BD Censo 2010

In [5]:
import glob

path = './../../samplerCensoARG/data/censo_samples'
# frac = 0.005
# files = glob.glob(path + '/*table_f'+str(frac)+'*_ARGa.csv')
files = glob.glob(path + '/*table_f*_ARGb.csv')
files = sorted(files)
print(files)


['./../../samplerCensoARG/data/censo_samples/table_f0.005_2003_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2004_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2005_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2006_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2007_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2008_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2009_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2010_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2011_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2012_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2013_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2014_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_f0.005_2015_ARGb.csv', './../../samplerCensoARG/data/censo_samples/table_

### Guardar muestras de Censo Adaptadas a EPH

In [6]:
from pathlib import Path
# pip install humanize
import humanize

# Path a donde guardar las tablas de sampleo de Censo adaptado a Modelo Encuestador
# (Tener en cuenta que pueden ocupar bastante espacio en disco)
out_path = '/media/matias/Elements/suite/yr_samples/'

for f in files:
    # Cargamos
    table = pd.read_csv(f)
    
    # frac = f.split('_')[-3][1:]
    yr = f.split('_')[-2]
    table['ANO4'] = int(yr)

    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    table = adapta_muestras_a_EPH(table)
    
    # Guardamos
    out_file = out_path + Path(f).stem + '_EPHedit.csv'
    table.to_csv(out_file, index = False)  # Copias en carpeta yr_samples, en nuestra carpeta de indice de pobreza

    # Imprimir tamaño de archivo
    size = os.path.getsize(out_file)
    human_readable_size = humanize.naturalsize(size)
    print(f"Table saved at {out_file} with size {human_readable_size}.")
    

Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2003_ARGb_EPHedit.csv with size 24.9 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2004_ARGb_EPHedit.csv with size 25.4 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2005_ARGb_EPHedit.csv with size 25.5 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2006_ARGb_EPHedit.csv with size 26.1 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2007_ARGb_EPHedit.csv with size 26.6 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2008_ARGb_EPHedit.csv with size 26.7 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2009_ARGb_EPHedit.csv with size 26.9 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2010_ARGb_EPHedit.csv with size 27.4 MB.
Table saved at /media/matias/Elements/suite/yr_samples/table_f0.005_2011_ARGb_EPHedit.csv with size 27.8 MB.
Table saved at /med